# Permutation Tests

In [1]:
# boilerplate
%matplotlib inline
import math
import numpy as np
import scipy as sp
import pandas as pd
from scipy import stats  # distributions
from scipy import special # special functions
from scipy import random # random variables, distributions, etc.
from scipy.optimize import brentq
from scipy.stats import (binom, hypergeom)
import matplotlib.pyplot as plt
from ipywidgets import widgets

khazan_fn = './Data/khazanEtal20.csv'

In [2]:
df = pd.read_csv(khazan_fn)
df.head()

,ID,Class,Gender,Age,number.online.courses.taken,TA,Question,Likert.Score,Theme,ScoreNegPos
0,1,3,female,21,3,Jesse,Facilitated.learning,4,teaching,1
1,1,3,female,21,3,Jesse,Provided.helpful.feedback,5,teaching,2
2,1,3,female,21,3,Jesse,Is.an.expert,4,knowledge,1
3,1,3,female,21,3,Jesse,Graded.in.a.timely.manner,5,professional,2
4,1,3,female,21,3,Jesse,Graded.Fairly,5,teaching,2


In [3]:
qs = df['Question'].unique()

In [4]:
for q in qs:
    mask = df['Question'] == q
    df.loc[mask,q] = df[mask]['Likert.Score']

In [5]:
df = df.drop(columns=['Class','Age','number.online.courses.taken','Question','Likert.Score',\
         'Theme','ScoreNegPos'])
df = df.set_index('ID')
df['Did.NOT.respond.to.email.promptly'] = -df['Did.NOT.respond.to.email.promptly']

In [6]:
df.head()

,Gender,TA,Facilitated.learning,Provided.helpful.feedback,Is.an.expert,Graded.in.a.timely.manner,Graded.Fairly,Did.NOT.respond.to.email.promptly,Knowledgable.of.course.content,Helpful.feedback.vias.Canvas.discussion,Consistently.fulfilled.responsibilities,Considerate.in.communication,Treated.me.with.respect,Enthusiastic,Professional,TA.again
ID,,,,,,,,,,,,,,,,
1,female,Jesse,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,female,Jesse,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,female,Jesse,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,female,Jesse,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,female,Jesse,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
agg_dict = {'TA': 'first'}
for q in qs:
    agg_dict[q] = np.nansum
scores = df.groupby('ID').agg(agg_dict).reset_index()
scores.head()

,ID,TA,Facilitated.learning,Provided.helpful.feedback,Is.an.expert,Graded.in.a.timely.manner,Graded.Fairly,Did.NOT.respond.to.email.promptly,Knowledgable.of.course.content,Helpful.feedback.vias.Canvas.discussion,Consistently.fulfilled.responsibilities,Considerate.in.communication,Treated.me.with.respect,Enthusiastic,Professional,TA.again
0,1,Jesse,4.0,5.0,4.0,5.0,5.0,-5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0
1,2,Jesse,4.0,5.0,5.0,4.0,4.0,-2.0,4.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0
2,3,Jesse,5.0,5.0,5.0,5.0,5.0,-5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,4,Jesse,4.0,4.0,4.0,4.0,4.0,-4.0,5.0,4.0,4.0,4.0,5.0,4.0,5.0,4.0
4,5,Emily,4.0,4.0,3.0,4.0,2.0,-4.0,3.0,3.0,5.0,3.0,3.0,3.0,4.0,3.0


In [8]:
mask = (scores['TA'] == 'Jesse')
jesse = scores.loc[mask].copy()
emily = scores.loc[~mask].copy()
emily.head()

,ID,TA,Facilitated.learning,Provided.helpful.feedback,Is.an.expert,Graded.in.a.timely.manner,Graded.Fairly,Did.NOT.respond.to.email.promptly,Knowledgable.of.course.content,Helpful.feedback.vias.Canvas.discussion,Consistently.fulfilled.responsibilities,Considerate.in.communication,Treated.me.with.respect,Enthusiastic,Professional,TA.again
4,5,Emily,4.0,4.0,3.0,4.0,2.0,-4.0,3.0,3.0,5.0,3.0,3.0,3.0,4.0,3.0
5,6,Emily,3.0,4.0,3.0,4.0,4.0,-3.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,4.0
6,7,Emily,4.0,5.0,4.0,5.0,3.0,-5.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,3.0
10,11,Emily,3.0,3.0,4.0,3.0,2.0,-3.0,3.0,3.0,5.0,5.0,5.0,5.0,3.0,1.0
11,12,Emily,5.0,5.0,5.0,5.0,5.0,-5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [9]:
# there were 11 nonresponders for "Emily" and 10 for "Jesse"
emily_missing = {'ID':0, 'TA':'Emily' }
jesse_missing = {'ID':0, 'TA':'Jesse' }
for q in qs:
    emily_missing[q] = np.nan
    jesse_missing[q] = np.nan
for i in range(11):
    emily = emily.append(emily_missing, ignore_index=True)
for i in range(10):
    jesse = jesse.append(jesse_missing, ignore_index=True)
print(emily.describe(), jesse.describe())

               ID  Facilitated.learning  Provided.helpful.feedback  \
count   66.000000             55.000000                  55.000000   
mean    48.878788              4.018182                   4.200000   
std     35.912507              1.146507                   1.128749   
min      0.000000              1.000000                   1.000000   
25%     13.750000              3.500000                   4.000000   
50%     53.000000              4.000000                   5.000000   
75%     76.250000              5.000000                   5.000000   
max    111.000000              5.000000                   5.000000   

       Is.an.expert  Graded.in.a.timely.manner  Graded.Fairly  \
count     55.000000                  55.000000      55.000000   
mean       4.018182                   4.309091       3.981818   
std        1.113734                   0.920401       1.146507   
min        1.000000                   1.000000       1.000000   
25%        3.500000                   4.0000

In [10]:
from scipy.stats import norm, rankdata
from cryptorandom.sample import random_sample
from cryptorandom.cryptorandom import SHA256
from permute.utils import get_prng, permute
prng = SHA256(1234567890)

def abs_mean_diff(x, y):
    return np.abs(np.nanmean(x)-np.nanmean(y))

def mean_diff(x, y):
    return np.nanmean(x)-np.nanmean(y)

def sim_npc(X, Y, cols, test_fn, combine="fisher", prng=None, reps=int(10**4), verbose=False):
    ts = {}
    tv = {}
    ps = {}
    XY = pd.concat([X,Y])
    nx = len(X[cols[0]])
    n  = len(XY[cols[0]])
    all_i = set(range(n))
    for c in cols:
        ts[c] = test_fn(X[c], Y[c])
        tv[c] = []
    if verbose:
        print('\nn: {} nx: {} ts:{}\n'.format(n, nx, ts))
    for i in range(reps):
        inx = random_sample(n, size=nx, replace=False, prng=prng) 
        iny = list(all_i - set(inx))  
        for c in cols:
            tv[c].append(test_fn(XY.iloc[inx][c], XY.iloc[iny][c]))
        if verbose and i%(int(reps/10)) == 0:
            print(i, [(np.sum(np.array(tv[c]) >= ts[c]) + 1)/(i+2) for c in cols]) 
    for c in cols:
        ps[c] = (np.sum(np.array(tv[c]) >= ts[c]) + 1)/(reps+1) 
    dist = np.array([tv[c] for c in cols]).T
    dist = np.append(dist, np.array([ts[c] for c in cols], ndmin=2), axis=0)
    p = npc(np.array([ps[c] for c in cols]), dist, combine=combine)
    return p, ts, ps

def npc(pvalues, distr, combine="fisher", plus1=True):
    r"""
    Combines p-values from individual partial test hypotheses $H_{0i}$ against
    $H_{1i}$, $i=1,\dots,n$ to test the global null hypothesis
    .. math:: \cap_{i=1}^n H_{0i}
    against the alternative
    .. math:: \cup_{i=1}^n H_{1i}
    using an omnibus test statistic.
    Parameters
    ----------
    pvalues : array_like
        Array of p-values to combine
    distr : array_like
        Array of dimension [B, n] where B is the number of permutations and n is
        the number of partial hypothesis tests. The $i$th column of distr contains
        the simulated null distribution of the $i$th test statistic under $H_{0i}$.
    combine : {'fisher', 'liptak', 'tippett'} or function
        The combining function to use. Default is "fisher".
        Valid combining functions must take in p-values as their argument and be
        monotonically decreasing in each p-value.
    plus1 : bool
        flag for whether to add 1 to the numerator and denominator of the
        p-value based on the empirical permutation distribution. 
        Default is True.
    Returns
    -------
    float
        A single p-value for the global test
    """
    n = len(pvalues)
    B = distr.shape[0]
    if n < 2:
        raise ValueError("One p-value: nothing to combine!")
    if n != distr.shape[1]:
        raise ValueError("Mismatch in number of p-values and size of distr")

    combine_library = {
        "fisher": fisher,
        "liptak": liptak,
        "tippett": tippett
    }
    if callable(combine):
        if not check_combfunc_monotonic(pvalues, combine):
            raise ValueError(
                "Bad combining function: must be monotonically decreasing in each p-value")
        combine_func = combine
    else:
        combine_func = combine_library[combine]

    # Convert test statistic distribution to p-values
    combined_stat_distr = [0] * B
    pvalues_from_distr = np.zeros((B, n))
    for j in range(n):
        pvalues_from_distr[:, j] = 1 - rankdata(distr[:, j], method="min")/(plus1+B) + (1 + plus1)/(plus1+B)
    if combine == "liptak":
        toobig = np.where(pvalues_from_distr >= 1)
        pvalues_from_distr[toobig] = 1 - np.finfo(float).eps
    combined_stat_distr = np.apply_along_axis(
        combine_func, 1, pvalues_from_distr)

    observed_combined_stat = combine_func(pvalues)
    return (plus1 + np.sum(combined_stat_distr >= observed_combined_stat)) / (plus1+B)

def fisher(pvalues):
    r"""
    Apply Fisher's combining function
    .. math:: -2 \sum_i \log(p_i)
    Parameters
    ----------
    pvalues : array_like
        Array of p-values to combine
    Returns
    -------
    float
        Fisher's combined test statistic
    """
    return -2*np.log(np.prod(pvalues))

def liptak(pvalues):
    r"""
    Apply Liptak's combining function
    .. math:: \sum_i \Phi^{-1}(1-p_i)
    where $\Phi^{-1}$ is the inverse CDF of the standard normal distribution.
    Parameters
    ----------
    pvalues : array_like
        Array of p-values to combine
    Returns
    -------
    float
        Liptak's combined test statistic
    """
    return np.sum(norm.ppf(1 - pvalues))


def tippett(pvalues):
    r"""
    Apply Tippett's combining function
    .. math:: \max_i \{1-p_i\}
    Parameters
    ----------
    pvalues : array_like
        Array of p-values to combine
    Returns
    -------
    float
        Tippett's combined test statistic
    """
    return np.max(1 - pvalues)

In [23]:
sim_npc(jesse, emily, qs, abs_mean_diff, combine="fisher", prng=prng, \
        reps=int(10**4), verbose=True)


n: 136 nx: 70 ts:{'Facilitated.learning': 0.051515151515151736, 'Provided.helpful.feedback': 0.06666666666666643, 'Is.an.expert': 0.01818181818181852, 'Graded.in.a.timely.manner': 0.17424242424242387, 'Graded.Fairly': 0.0848484848484845, 'Did.NOT.respond.to.email.promptly': 0.15757575757575726, 'Knowledgable.of.course.content': 0.0333333333333341, 'Helpful.feedback.vias.Canvas.discussion': 0.13333333333333375, 'Consistently.fulfilled.responsibilities': 0.06666666666666643, 'Considerate.in.communication': 0.0242424242424244, 'Treated.me.with.respect': 0.10303030303030347, 'Enthusiastic': 0.12575757575757596, 'Professional': 0.15000000000000036, 'TA.again': 0.15000000000000036}

0 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 1.0]
1000 [0.8303393213572854, 0.7255489021956087, 0.8892215568862275, 0.27245508982035926, 0.7125748502994012, 0.4540918163672655, 0.8293413173652695, 0.49101796407185627, 0.6307385229540918, 0.8982035928143712, 0.4600798403193613, 0.4900199600

(0.9065186962607479,
 {'Facilitated.learning': 0.051515151515151736,
  'Provided.helpful.feedback': 0.06666666666666643,
  'Is.an.expert': 0.01818181818181852,
  'Graded.in.a.timely.manner': 0.17424242424242387,
  'Graded.Fairly': 0.0848484848484845,
  'Did.NOT.respond.to.email.promptly': 0.15757575757575726,
  'Knowledgable.of.course.content': 0.0333333333333341,
  'Helpful.feedback.vias.Canvas.discussion': 0.13333333333333375,
  'Consistently.fulfilled.responsibilities': 0.06666666666666643,
  'Considerate.in.communication': 0.0242424242424244,
  'Treated.me.with.respect': 0.10303030303030347,
  'Enthusiastic': 0.12575757575757596,
  'Professional': 0.15000000000000036,
  'TA.again': 0.15000000000000036},
 {'Facilitated.learning': 0.835016498350165,
  'Provided.helpful.feedback': 0.7364263573642635,
  'Is.an.expert': 0.8813118688131187,
  'Graded.in.a.timely.manner': 0.2696730326967303,
  'Graded.Fairly': 0.7234276572342766,
  'Did.NOT.respond.to.email.promptly': 0.47305269473052697,

In [22]:
sim_npc(jesse, emily, qs, mean_diff, combine="tippett", prng=prng, \
        reps=int(10**4), verbose=True)


n: 136 nx: 66 ts:{'Facilitated.learning': 0.051515151515151736, 'Provided.helpful.feedback': -0.06666666666666643, 'Is.an.expert': 0.01818181818181852, 'Graded.in.a.timely.manner': -0.17424242424242387, 'Graded.Fairly': -0.0848484848484845, 'Did.NOT.respond.to.email.promptly': 0.15757575757575726, 'Knowledgable.of.course.content': 0.0333333333333341, 'Helpful.feedback.vias.Canvas.discussion': -0.13333333333333375, 'Consistently.fulfilled.responsibilities': 0.06666666666666643, 'Considerate.in.communication': -0.0242424242424244, 'Treated.me.with.respect': -0.10303030303030347, 'Enthusiastic': 0.12575757575757596, 'Professional': -0.15000000000000036, 'TA.again': -0.15000000000000036}

0 [0.5, 1.0, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 1.0, 0.5]
1000 [0.43812375249500995, 0.6526946107784432, 0.45209580838323354, 0.8792415169660679, 0.6736526946107785, 0.23552894211576847, 0.43313373253493015, 0.7644710578842315, 0.312375249500998, 0.5898203592814372, 0.811377245508982, 0.24

(0.7869426114777045,
 {'Facilitated.learning': 0.051515151515151736,
  'Provided.helpful.feedback': -0.06666666666666643,
  'Is.an.expert': 0.01818181818181852,
  'Graded.in.a.timely.manner': -0.17424242424242387,
  'Graded.Fairly': -0.0848484848484845,
  'Did.NOT.respond.to.email.promptly': 0.15757575757575726,
  'Knowledgable.of.course.content': 0.0333333333333341,
  'Helpful.feedback.vias.Canvas.discussion': -0.13333333333333375,
  'Consistently.fulfilled.responsibilities': 0.06666666666666643,
  'Considerate.in.communication': -0.0242424242424244,
  'Treated.me.with.respect': -0.10303030303030347,
  'Enthusiastic': 0.12575757575757596,
  'Professional': -0.15000000000000036,
  'TA.again': -0.15000000000000036},
 {'Facilitated.learning': 0.41985801419858015,
  'Provided.helpful.feedback': 0.6459354064593541,
  'Is.an.expert': 0.45835416458354167,
  'Graded.in.a.timely.manner': 0.8809119088091191,
  'Graded.Fairly': 0.6594340565943405,
  'Did.NOT.respond.to.email.promptly': 0.2498750

In [20]:
from scipy.stats import chi2
chi2.sf(-2*np.log(0.08*1.0e-5),df=4)

1.203092328742278e-05